In [1]:
import pandas as pd

**Задание 1**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
ratings = pd.read_csv('ml-25m/ratings.csv')

def rating_classification(data):
    if data <= 2:
        return 'низкий'
    elif data <= 4:
        return 'средний'
    else:
        return 'высокий'
    
ratings['class'] = ratings['rating'].apply(rating_classification)
ratings

,userId,movieId,rating,timestamp,class
0,1,296,5.0,1147880044,высокий
1,1,306,3.5,1147868817,средний
2,1,307,5.0,1147868828,высокий
3,1,665,5.0,1147878820,высокий
4,1,899,3.5,1147868510,средний
...,...,...,...,...,...
25000090,162541,50872,4.5,1240953372,высокий
25000091,162541,55768,2.5,1240951998,средний
25000092,162541,56176,2.0,1240950697,низкий
25000093,162541,58559,4.0,1240953434,средний


**Задание 2**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

```
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

In [3]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

data = pd.read_csv('keywords.csv')
data

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
...,...,...
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688


In [4]:
def parse_region(raw):
    for region, cities in geo_data.items():
        for city in cities:
            if city in raw['keyword']:
                return region
    return 'undefined'

data['region'] = data.apply(parse_region, axis=1)
data[data['region'] != 'undefined']

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
...,...,...,...
99590,авито ярославль автомобили с пробегом,3673,Центр
99634,северпост новости мурманской области,3606,Северо-Запад
99808,полармед мурманск запись на прием,3719,Северо-Запад
99890,яндекс метро москва,4548,Центр


***Задание 3 (бонусное)**

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [5]:
ratings = pd.read_csv('ml-25m/ratings.csv')
movies = pd.read_csv('ml-25m/movies.csv')
ratings = ratings.merge(movies, on='movieId')

years = list(range(1950, 2011))

In [7]:
%%time

def get_production_year(title):
    for year in years:
        if str(year) in title:
            return year
    return 1900

ratings['year'] = ratings['title'].apply(get_production_year)
ratings_by_years = ratings[['rating', 'year']] \
    .groupby(by='year') \
    .mean() \
    .sort_values('rating', ascending=False)
ratings_by_years

CPU times: user 3min 54s, sys: 2.22 s, total: 3min 57s
Wall time: 3min 58s


,rating
year,
1957,3.999492
1954,3.985693
1972,3.956477
1962,3.929445
1952,3.895315
...,...
1990,3.435736
2005,3.434087
2000,3.420897
